In [1]:
import pandas as pd
import plistlib

In [2]:
gbv_call_data_2020=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-2020.csv')

In [3]:
gbv_call_data_2021=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-2021_0.csv')

In [4]:
gbv_call_data_2022_1=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-1ersemestre-2022.csv')

1. Apped all the dataframes from 2020, 2021 and 2022 in one Datafrmae

In [5]:
gvb_calls_data=gbv_call_data_2020.append([gbv_call_data_2021, gbv_call_data_2022_1])
gvb_calls_data

/var/folders/pw/szr0m5sn17vgnpycyzj91y5r0000gn/T/ipykernel_83009/1911846326.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gvb_calls_data=gbv_call_data_2020.append([gbv_call_data_2021, gbv_call_data_2022_1])


,fecha,prov_residencia_persona_en_situacion_violencia,genero_persona_en_situacion_de_violencia,edad_persona_en_situacion_de_violencia,pais_nacimiento_persona_en_situacion_de_violencia,tipo_de_violencia_fisica,tipo_de_violencia_psicologica,tipo_de_violencia_sexual,tipo_de_violencia_economica_y_patrimonial,tipo_de_violencia_simbolica,tipo_de_violencia_domestica,modalidad_de_violencia_institucional,modalidad_de_violencia_laboral,modalidad_violencia_contra_libertad_reproductiva,modalidad_de_violencia_obstetrica,modalidad_de_violencia_mediatica,modalidad_de_violencia_otras,vinculo_con_la_persona_agresora,genero_de_la_persona_agresora
0,2020-01-01,Tucumán,Mujer,NaN,NaN,Si,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,Si,Si,No,No,No,Si,No,No,No,No,No,No,Ex pareja,Varon
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,Si,Si,No,No,No,Si,No,No,No,No,No,No,Ex pareja,Varon
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,No,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,Si,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,2022-06-30,Buenos Aires,Mujer,NaN,NaN,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Pareja,Varon
12394,2022-06-30,Buenos Aires,Mujer,67.0,Argentina,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Pareja,Varon
12395,2022-06-30,Buenos Aires,Mujer,52.0,Uruguaya,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Ex pareja,Varon
12396,2022-06-30,Buenos Aires,Mujer,NaN,Argentina,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Ex pareja,Varon


In [6]:
gvb_calls_data

,fecha,prov_residencia_persona_en_situacion_violencia,genero_persona_en_situacion_de_violencia,edad_persona_en_situacion_de_violencia,pais_nacimiento_persona_en_situacion_de_violencia,tipo_de_violencia_fisica,tipo_de_violencia_psicologica,tipo_de_violencia_sexual,tipo_de_violencia_economica_y_patrimonial,tipo_de_violencia_simbolica,tipo_de_violencia_domestica,modalidad_de_violencia_institucional,modalidad_de_violencia_laboral,modalidad_violencia_contra_libertad_reproductiva,modalidad_de_violencia_obstetrica,modalidad_de_violencia_mediatica,modalidad_de_violencia_otras,vinculo_con_la_persona_agresora,genero_de_la_persona_agresora
0,2020-01-01,Tucumán,Mujer,NaN,NaN,Si,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,Si,Si,No,No,No,Si,No,No,No,No,No,No,Ex pareja,Varon
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,Si,Si,No,No,No,Si,No,No,No,No,No,No,Ex pareja,Varon
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,No,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,Si,Si,No,No,No,Si,No,No,No,No,No,No,Pareja,Varon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,2022-06-30,Buenos Aires,Mujer,NaN,NaN,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Pareja,Varon
12394,2022-06-30,Buenos Aires,Mujer,67.0,Argentina,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Pareja,Varon
12395,2022-06-30,Buenos Aires,Mujer,52.0,Uruguaya,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Ex pareja,Varon
12396,2022-06-30,Buenos Aires,Mujer,NaN,Argentina,Si,Si,No,No,Si,Si,No,No,No,No,No,No,Ex pareja,Varon


# --------Cleaning---------

In [7]:
gvb_calls_data.isna().sum()

fecha                                                    0
prov_residencia_persona_en_situacion_violencia         974
genero_persona_en_situacion_de_violencia              1274
edad_persona_en_situacion_de_violencia               12991
pais_nacimiento_persona_en_situacion_de_violencia    22772
tipo_de_violencia_fisica                                 0
tipo_de_violencia_psicologica                            0
tipo_de_violencia_sexual                                 0
tipo_de_violencia_economica_y_patrimonial                0
tipo_de_violencia_simbolica                              0
tipo_de_violencia_domestica                              0
modalidad_de_violencia_institucional                     0
modalidad_de_violencia_laboral                           0
modalidad_violencia_contra_libertad_reproductiva         0
modalidad_de_violencia_obstetrica                        0
modalidad_de_violencia_mediatica                         0
modalidad_de_violencia_otras                            

Drop "pais_nacimiento_persona_en_situacion_de_violencia" column because more than 30% of mv

In [8]:
gvb_calls_data.drop(columns=['pais_nacimiento_persona_en_situacion_de_violencia'], inplace=True)

In [9]:
# gvb_calls_data=gvb_calls_data.where(lambda row : row['genero_de_la_persona_agresora']!=0).dropna()
# print(type(gvb_calls_data))

Cleaning mv in the column "edad_persona_en_situacion_de_violencia"

In [10]:
gvb_calls_data['edad_persona_en_situacion_de_violencia'].mean()

35.85057154408792